In [1]:
#from scipy.signal import savgol_filter
#import tensorflow as tf

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display
from pandas.plotting import autocorrelation_plot
from matplotlib import ticker
from pathlib import Path

In [2]:
#path_to_data = 'data\\Original data\\Run1\\run1_WM32_data.csv' # расположение всех измерений
#path_to_save_data = 'data\\Prepared data\\Run1\\X_data.xlsx' # куда сохранять X часть выборки

path_to_data = 'data\\Original data\\Run2\\run2_WM32_data.csv' # расположение всех измерений
path_to_save_data = 'data\\Prepared data\\Run2\\X_data.xlsx' # куда сохранять X часть выборки

In [3]:
def get_df(path: str):
    df = pd.read_csv(path, delimiter=';')
    return df

In [4]:
# Чтение данных из файла
first_df = get_df(path_to_data)

In [5]:
# Что прочли из файла
#display(first_df)

In [6]:
# Удаляем пустые столбцы
'''print(first_df.shape)
first_df = first_df.T
for col in first_df.columns:
    first_df = first_df.loc[~first_df[col].isin(['--'])]
first_df = first_df.T
print(first_df.shape)'''

"print(first_df.shape)\nfirst_df = first_df.T\nfor col in first_df.columns:\n    first_df = first_df.loc[~first_df[col].isin(['--'])]\nfirst_df = first_df.T\nprint(first_df.shape)"

In [7]:
# Удаляем лишние колонки, назначаем новый индекс
first_df = first_df.drop(['position'], axis=1)
first_df['row'] = first_df['row'].astype(int)
first_df = first_df.set_index('row')

In [8]:
# Что вышло после начальной обработки
#display(first_df)

In [9]:
def prepare_row_dataframe(df):
    temp_row_df = pd.DataFrame()

    for col_name in list(df.columns):

        temp =  df[col_name].str.split(pat=':', expand=True, n=2)
        series_list = [temp[col] for col in temp]

        temp = pd.concat(series_list, ignore_index=True)
        temp = temp.to_frame()
        temp.columns = [col_name]
        temp_row_df = pd.concat([temp_row_df,temp],axis=1)

    return temp_row_df

In [10]:
def prepare_dataframe(df):
    prepared_df = pd.DataFrame()
    # prepared_df - обработанный датафрейм
    # row_name - имя строки обрабатываемой в отдельной итерации цикла
    # rows_num - кол-во строк в датафрейме, полученном после разделения по ',' (кол-во измерений 30-31)
    # row_df - датафрейм полученный после expand строки начального датафрейма 
    
    # перебор по строкам начального датафрейма
    for i in df.index.values.tolist():
        
        row_df = df.iloc[i].str.split(pat=',', expand=True).T
        
        rows_num = row_df.shape[0]
        if (rows_num < 32):
            for j in range(32 - rows_num):
                row_df.loc[len(row_df.index)] = '0:0'

        row_df = prepare_row_dataframe(row_df)
        
        row_df.index = pd.MultiIndex.from_product([[i], row_df.index.values.tolist()], names=["Row", "Data point"])

        prepared_df = pd.concat([prepared_df, row_df])
    
        #prepared_df.astype(float, errors='ignore')
    
    return prepared_df

In [11]:
prepared_df = prepare_dataframe(first_df)
#print(prepared_df.dtypes)
display(prepared_df)

detector_0 detector_1 detector_2 detector_3 detector_4  \
Row Data point                                                          
0   0                25.5       25.3       25.1       25.1       25.5   
    1                25.7       25.5       25.3       25.3       25.7   
    2                25.8       25.7       25.5       25.4       25.9   
    3                25.9       25.8       25.6       25.5         26   
    4                26.1       25.9       25.9       25.7       26.1   
...                   ...        ...        ...        ...        ...   
118 59            -39.598    -31.496    -38.781    -42.332    -37.094   
    60             35.327     31.496     39.192     39.598     35.327   
    61            -34.871    -36.661    -32.985    -35.327    -41.569   
    62            -30.984     35.327     33.466     32.496     40.398   
    63                  0          0          0          0          0   

               detector_5 detector_6 detector_7 detector_8 detector_9  ...  \
Row Data point                                                         ...   
0   0                25.5       25.1       25.8       25.8       25.8  ...   
    1                25.7       25.3       25.9       25.9         26  ...   
    2                25.9       25.4         26       26.1       26.1  ...   
    3                  26       25.5       26.1       26.2       26.3  ...   
    4                26.1       25.6       26.2       26.3       26.4  ...   
...                   ...        ...        ...        ...        ...  ...   
118 59             41.952    -38.781    -45.255    -43.081     42.332  ...   
    60            -35.327     37.094     44.542     41.183     34.871  ...   
    61            -39.598    -33.466    -37.094    -38.781    -41.183  ...   
    62             38.367     32.985    -36.222     36.661     39.192  ...   
    63                  0          0          0          0          0  ...   

               detector_390 detector_391 detector_392 detector_393  \
Row Data point                                                       
0   0                  24.8           25         24.8         24.7   
    1                  25.1         25.3         24.9         24.9   
    2                  25.2         25.4         25.1           25   
    3                  25.3         25.5         25.2         25.1   
    4                  25.5         25.7         25.3         25.3   
...                     ...          ...          ...          ...   
118 59               29.394      -37.947       38.367       38.367   
    60              -33.941       37.523      -31.496       31.496   
    61               30.984      -32.985      -35.777      -37.094   
    62              -28.844           32       33.466       34.409   
    63                    0            0            0            0   

               detector_394 detector_395 detector_396 detector_397  \
Row Data point                                                       
0   0                  24.7           20         24.7         24.7   
    1                  24.9         24.9         24.9         24.9   
    2                    25         25.1           25           25   
    3                  25.1         25.2         25.1         25.1   
    4                  25.3         25.3         25.3         25.3   
...                     ...          ...          ...          ...   
118 59              -36.222      -36.222          -40      -39.598   
    60               30.463       35.777       38.367       37.523   
    61               28.284       30.463      -35.777      -35.327   
    62              -29.933       30.463       33.466       33.941   
    63                    0            0            0            0   

               detector_398 detector_399  
Row Data point                            
0   0                    25           25  
    1                  25.2         25.2  
    2                  25.3         25.3  
    3        

In [12]:
# СОХРАНЯЕМ ДАТАСЕТ В ФАЙЛ
#prepared_df.to_excel(path_to_save_data)  

In [13]:
# Вырезаем интересующую строку из начального датафрейма
# Разделяем значения её ячеек по строкам (разделитель - ","). Считаем каждый датчик провел 31 измерение за 1 пуск 
# Выводим полученный датафрейм со столбцами - номерами датчиков и строками - номерами измерений
# В каждой ячейке пара время + амплитуда 

# Датафрейм со всеми измерениями без деления на время и амплитуду

'''row_df = df.iloc[0].str.split(pat=',', expand=True).T
rows_num = row_df.shape[0]

if (rows_num < 32):
    for i in range(32 - rows_num):
        row_df.loc[len(row_df.index)] = '0:0'
        
display(row_df)'''

"row_df = df.iloc[0].str.split(pat=',', expand=True).T\nrows_num = row_df.shape[0]\n\nif (rows_num < 32):\n    for i in range(32 - rows_num):\n        row_df.loc[len(row_df.index)] = '0:0'\n        \ndisplay(row_df)"

In [14]:
'''temp_row_df = pd.DataFrame()

for col_name in list(prepared_df.columns):
    
    temp = prepared_df[col_name].str.split(pat=':', expand=True, n=2)
    series_list = [temp[col] for col in temp]
    
    temp = pd.concat(series_list, ignore_index=True)
    temp = temp.to_frame()
    temp.columns = [col_name]
    temp_row_df = pd.concat([temp_row_df,temp],axis=1)
    
display(temp_row_df)'''

"temp_row_df = pd.DataFrame()\n\nfor col_name in list(prepared_df.columns):\n    \n    temp = prepared_df[col_name].str.split(pat=':', expand=True, n=2)\n    series_list = [temp[col] for col in temp]\n    \n    temp = pd.concat(series_list, ignore_index=True)\n    temp = temp.to_frame()\n    temp.columns = [col_name]\n    temp_row_df = pd.concat([temp_row_df,temp],axis=1)\n    \ndisplay(temp_row_df)"

In [15]:
'''prepared_df = pd.DataFrame()

# перебор по строкам начального датафрейма
for i in range(1):
    row_df = df.iloc[i].str.split(pat=',', expand=True).T
    rows_num = row_df.shape[0]

    if (rows_num < 32):
        for j in range(32 - rows_num):
            row_df.loc[len(row_df.index)] = '0:0'
    
    #row_df.index = pd.MultiIndex.from_product([[i], row_df.index.values.tolist()], names=["Row", "Data point"])
    
    prepared_df = pd.concat([prepared_df, row_df])
    
display(prepared_df)'''

'prepared_df = pd.DataFrame()\n\n# перебор по строкам начального датафрейма\nfor i in range(1):\n    row_df = df.iloc[i].str.split(pat=\',\', expand=True).T\n    rows_num = row_df.shape[0]\n\n    if (rows_num < 32):\n        for j in range(32 - rows_num):\n            row_df.loc[len(row_df.index)] = \'0:0\'\n    \n    #row_df.index = pd.MultiIndex.from_product([[i], row_df.index.values.tolist()], names=["Row", "Data point"])\n    \n    prepared_df = pd.concat([prepared_df, row_df])\n    \ndisplay(prepared_df)'

In [16]:
'''s1 = pd.Series(['a', 'b','c','d'])
s2 = pd.Series(['c', 'd','c','d'])

temp = pd.concat([s1, s2], keys=('s1','s2'),
    names=['Series name', 'Row ID'])

print(type(temp))

display(temp)'''

"s1 = pd.Series(['a', 'b','c','d'])\ns2 = pd.Series(['c', 'd','c','d'])\n\ntemp = pd.concat([s1, s2], keys=('s1','s2'),\n    names=['Series name', 'Row ID'])\n\nprint(type(temp))\n\ndisplay(temp)"

In [17]:
'''s1 = pd.DataFrame([['A', 1, 'cat'], ['B', 2, 'dog']], columns=['letter', 'number', 'animal'])
s2 = pd.DataFrame([['C', 2, 'owe'], ['D', 4, 'fish']], columns=['letter', 'number', 'animal'])

temp = pd.concat([s1, s2], keys=('s1','s2'),
    names=['Dataframe name', 'Row ID'])

print(len(temp.index))
print(type(temp))

display(temp)'''

"s1 = pd.DataFrame([['A', 1, 'cat'], ['B', 2, 'dog']], columns=['letter', 'number', 'animal'])\ns2 = pd.DataFrame([['C', 2, 'owe'], ['D', 4, 'fish']], columns=['letter', 'number', 'animal'])\n\ntemp = pd.concat([s1, s2], keys=('s1','s2'),\n    names=['Dataframe name', 'Row ID'])\n\nprint(len(temp.index))\nprint(type(temp))\n\ndisplay(temp)"

In [18]:
'''s3 = pd.DataFrame([['E', 5, 'BABA'], ['F', 6, 'DADA']], columns=['letter', 'number', 'animal'], index = pd.Index([0,1],name='Row ID'))

display(s3)

s3.index = pd.MultiIndex.from_product([['s3'],s3.index.values.tolist()], names=["Dataframe name", "Row ID"])

display(s3)'''

's3 = pd.DataFrame([[\'E\', 5, \'BABA\'], [\'F\', 6, \'DADA\']], columns=[\'letter\', \'number\', \'animal\'], index = pd.Index([0,1],name=\'Row ID\'))\n\ndisplay(s3)\n\ns3.index = pd.MultiIndex.from_product([[\'s3\'],s3.index.values.tolist()], names=["Dataframe name", "Row ID"])\n\ndisplay(s3)'

In [19]:
'''temp = pd.concat([temp, s3])

display(temp)'''

'temp = pd.concat([temp, s3])\n\ndisplay(temp)'